<a href="https://colab.research.google.com/github/aishwaryaanaidu/birdcall-identification/blob/main/test_train_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install pydub

In [4]:
pip install flask_ngrok

In [5]:
pip install flask_cors

In [6]:
import librosa, librosa.display
import os
from matplotlib import pyplot as plt
import IPython.display as ipd
import numpy as np
import torch
# matplotlib inline
from pydub import AudioSegment
# import FigureCanvasAgg as FigureCanvas
from os import path
import subprocess
import csv
from csv import writer

#Keras imports
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.models import model_from_json
import pandas as pd
from PIL import Image
from keras.utils import np_utils
from sklearn import preprocessing
from matplotlib.image import imread

imheight = 360
imwidth = 540

le = preprocessing.LabelEncoder()

In [37]:
train_list = []
directory = "drive/MyDrive/Colab Notebooks/IndianaBirds"
for bird in os.listdir(directory):
  for filename in os.listdir(directory + "/" + bird):
    train_list.append([filename.split(".")[0] + ".png", bird])
file1 = open('dabba.csv', 'w+', newline = '')
with file1:
    write = csv.writer(file1)
    write.writerows(train_list)

In [38]:
filename = 'drive/MyDrive/Colab Notebooks/test_data1.csv'

df = pd.read_csv("dabba.csv")
trl2 = df.values.tolist()

testt = []
with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        if row in trl2:
          testt.append(row)
file1 = open('dabba1.csv', 'w+', newline = '')
with file1:
    write = csv.writer(file1)
    write.writerows(testt)

In [ ]:
def create_spectrogram():
    # imheight = 50
    # imwidth = 34
    train_dir = "drive/MyDrive/Colab Notebooks/Spectrograms/train_spectrograms"
    test_dir = "drive/MyDrive/Colab Notebooks/Spectrograms/test_spectrograms"
    ####
    os.mkdir(train_dir)
    os.mkdir(test_dir)
    ####
    train_list = [["path", "label"]]
    test_list = [["path", "label"]]
    for bird in os.listdir(directory)[:5]:
        dir_len = len(os.listdir(directory + "/" + bird)[:5])
        train_count = round(dir_len * 0.7)
        count = 1
        for filename in os.listdir(directory + "/" + bird)[:5]:
            if count <= train_count:
                new_dir = train_dir
                train_list.append([new_dir + "/" + filename.split(".")[0] + ".png", bird])
            else:
                new_dir = test_dir
                test_list.append([new_dir + "/" + filename.split(".")[0] + ".png", bird])
            src = directory + "/" + bird + "/" + filename
            save_path = new_dir + "/" + filename.split(".")[0] + ".png"
            # print(src)
            # print(save_path)

            dst = "test.wav"
            # params
            sampling = 21952
            hop_length = 245
            n_mels = 224
            n_fft = 892
            win_length = n_fft
            plt.rcParams["figure.figsize"] = [7.50, 5.00]
            plt.rcParams["figure.autolayout"] = True

            # convert mp3 to wav
            sound = AudioSegment.from_mp3(src)
            sound.export(dst, format="wav")
            signal, Fs = librosa.load(dst, sr=21952)

            # Load signal and plot
            signal,Fs = librosa.load(dst,sr=sampling,mono=True,res_type="kaiser_fast")
            plt.axis('off')

            # Create spectrogram and plot
            spectr = librosa.feature.melspectrogram(signal, sr=sampling, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length,
                                                    win_length=win_length, fmin=300)
            log_spectr = librosa.amplitude_to_db(spectr)

            librosa.display.specshow(log_spectr, sr=sampling, hop_length=hop_length)
            ####
            plt.savefig(save_path)
            
            gray = Image.open(save_path).convert('L')
            gray.save(save_path)
            ####
            

            count += 1
    # return train_list, test_list

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def list_to_csv(train_lis, test_lis):
    file1 = open('train.csv', 'w+', newline = '')
    file2 = open('test.csv', 'w+', newline = '')
    with file1:
        write = csv.writer(file1)
        write.writerows(train_lis)
    with file2:
        write = csv.writer(file2)
        write.writerows(test_lis)

In [ ]:
def img_to_numpy_array(path):
    return imread(path)

In [ ]:
def create_train_test(trl, tel):
    train = pd.read_csv("train.csv")
    test = pd.read_csv("test.csv")
    # y_test = np.zeros(len(tel[1:]), str)
    # y_train = np.zeros(len(trl[1:]), str)
    y_test = np.empty((len(tel[1:]), 1), dtype=np.dtype('U100'))
    y_train = np.empty((len(trl[1:]), 1), dtype=np.dtype('U100'))
    x_train = np.zeros((len(trl[1:]), imheight, imwidth))
    x_test = np.zeros((len(tel[1:]), imheight, imwidth))

    for i, f in enumerate(tel[1:]):
        y_test[i]     = f[1]
        x_test[i,:,:] = img_to_numpy_array(f[0])

    for i, f in enumerate(trl[1:]):
        y_train[i]     = f[1]
        x_train[i,:,:] = img_to_numpy_array(f[0])
    return x_train, y_train, x_test, y_test

In [ ]:
def create_model(trl, tel):
    # train = pd.read_csv("train.csv")
    # test = pd.read_csv("test.csv")
    # x_train, y_train, x_test, y_test = train["path"], train["label"], test["path"], test["label"]
    x_train, y_train, x_test, y_test = create_train_test(trl, tel)

    print("Size of Training Data:", np.shape(x_train))
    print("Size of Training Labels:", np.shape(y_train))
    print("Size of Test Data:", np.shape(x_test))
    print("Size of Test Labels:", np.shape(y_test))
    
    # le = preprocessing.LabelEncoder()
    le.fit(["amerob", "blujay", "moudov", "norcar"])
    y_train = le.transform(y_train)
    y_test = le.transform(y_test)
    

    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)

    x_train = x_train.reshape(x_train.shape[0], imheight, imwidth, 1)
    x_test = x_test.reshape(x_test.shape[0], imheight, imwidth, 1)
    
    input_shape = (imheight, imwidth, 1)
    batch_size = 2
    epochs = 1

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.25))    
    model.add(Dense(128, activation='relu'))
#     model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
#     opt = keras.optimizers.Adam(learning_rate=0.01)

    model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
    print(model.summary())

    model.fit(x_train, y_train, batch_size=4, epochs=10, verbose=1, validation_data=(x_test, y_test))
    return model

In [18]:
directory = "drive/MyDrive/Colab Notebooks/IndianaBirds"
# trl, tel = create_spectrogram()
create_spectrogram()
train_df = pd.read_csv("drive/MyDrive/Colab Notebooks/train_data.csv")
trl = train_df.values.tolist()
test_df = pd.read_csv("drive/MyDrive/Colab Notebooks/test_data.csv")
tel = test_df.values.tolist()
# print(tel)
# list_to_csv(trl, tel)

[['XC351871.png', 'amerob'], ['XC351879.png', 'amerob'], ['XC353375.png', 'amerob'], ['XC361931.png', 'amerob'], ['XC364119.png', 'amerob'], ['XC364638.png', 'amerob'], ['XC369855.png', 'amerob'], ['XC374685.png', 'amerob'], ['XC377115.png', 'amerob'], ['XC390282.png', 'amerob'], ['XC398608.png', 'amerob'], ['XC402800.png', 'amerob'], ['XC406162.png', 'amerob'], ['XC418153.png', 'amerob'], ['XC418265.png', 'amerob'], ['XC421278.png', 'amerob'], ['XC423450.png', 'amerob'], ['XC423451.png', 'amerob'], ['XC425811.png', 'amerob'], ['XC385658.png', 'blujay'], ['XC385659.png', 'blujay'], ['XC385660.png', 'blujay'], ['XC385661.png', 'blujay'], ['XC390596.png', 'blujay'], ['XC391728.png', 'blujay'], ['XC391964.png', 'blujay'], ['XC392232.png', 'blujay'], ['XC394311.png', 'blujay'], ['XC395464.png', 'blujay'], ['XC413285.png', 'blujay'], ['XC418322.png', 'blujay'], ['XC421950.png', 'blujay'], ['XC425570.png', 'blujay'], ['XC433419.png', 'blujay'], ['XC434250.png', 'blujay'], ['XC436044.png', 'b

In [ ]:
#Save created model
def save_model_to_disk(model):
    # serialize model to JSON
    print("Hrrr")
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")

#Load saved model
def load_model_from_disk():
    print("Lrrrrr")
    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
    return loaded_model

In [ ]:
num_classes = 4
if(os.path.isfile('model.json')):
    m = load_model_from_disk()
else:
    m = create_model(trl, tel)
    save_model_to_disk(m)
# m = create_model(trl, tel)
print(m.summary())

Size of Training Data: (16, 360, 540)
Size of Training Labels: (16, 1)
Size of Test Data: (4, 360, 540)
Size of Test Labels: (4, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 358, 538, 32)      320       
                                                                 
 conv2d_1 (Conv2D)           (None, 356, 536, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 178, 268, 64)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 3053056)           0         
                                                                 
 dropout (Dropout)           (None, 3053056)           0         
                                                                 
 dense (Dense)               (None, 128)               390791296 
                                                        

In [ ]:
# from flask import Flask, jsonify, request
# from flask_ngrok import run_with_ngrok

# app = Flask(__name__)
# run_with_ngrok(app)

# @app.route('/')
# def hello():
#     temp = np.zeros((1, imheight, imwidth))
#     temp[0,:,:] = img_to_numpy_array("drive/MyDrive/Colab Notebooks/Spectrograms/test_spectr/XC133570.png")
#     temp.reshape(temp.shape[0], imheight, imwidth, 1)
#     output = np.array(m.predict(temp))
#     print(output)
#     # print(output[output.argmax()])
#     predictions_test = le.inverse_transform([output.argmax()])
#     print(predictions_test)
#     return predictions_test[0]

# if __name__ == "__main__":
#   app.run()


In [ ]:
def create_upload_spectrogram():
  upload_dir = "drive/MyDrive/Colab Notebooks/Spectrograms/uploads"
    ####
  os.mkdir(upload_dir)
  src = "drive/MyDrive/Colab Notebooks/uploads/upload.mp3"
  save_path = upload_dir + "/upload.png"
  # print(src)
  # print(save_path)

  dst = "test.wav"
  # params
  sampling = 21952
  hop_length = 245
  n_mels = 224
  n_fft = 892
  win_length = n_fft
  plt.rcParams["figure.figsize"] = [7.50, 5.00]
  plt.rcParams["figure.autolayout"] = True

  # convert mp3 to wav
  sound = AudioSegment.from_mp3(src)
  sound.export(dst, format="wav")
  signal, Fs = librosa.load(dst, sr=21952)

  # Load signal and plot
  signal,Fs = librosa.load(dst,sr=sampling,mono=True,res_type="kaiser_fast")
  plt.axis('off')

  # Create spectrogram and plot
  spectr = librosa.feature.melspectrogram(signal, sr=sampling, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length,
                                          win_length=win_length, fmin=300)
  log_spectr = librosa.amplitude_to_db(spectr)

  librosa.display.specshow(log_spectr, sr=sampling, hop_length=hop_length)
  ####
  plt.savefig(save_path)

  gray = Image.open(save_path).convert('L')
  gray.save(save_path)
  ####

In [ ]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS, cross_origin
from flask import jsonify

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'
app.config['UPLOAD_FOLDER'] = 'drive/MyDrive/Colab Notebooks/uploads'

run_with_ngrok(app)

@cross_origin()
@app.route('/', methods=['POST'])
def upload_file():
    uploaded_file = request.files['file']
    # print(uploaded_file.filename)
    # if uploaded_file.filename != '':
    #     uploaded_file.save(os.path.join(app.config['UPLOAD_FOLDER'], "upload.mp3"))

    # create_upload_spectrogram()
    # # return { "message": "success" }
    # temp = np.zeros((1, imheight, imwidth))
    # temp[0,:,:] = img_to_numpy_array("drive/MyDrive/Colab Notebooks/Spectrograms/uploads/upload.png")
    # temp.reshape(temp.shape[0], imheight, imwidth, 1)
    # output = np.array(m.predict(temp))
    # print(output)
    # # print(output[output.argmax()])
    # predictions_test = le.inverse_transform([output.argmax()])
    # print(predictions_test)
    data = { "predictions": "amerob"}
    return data

if __name__ == "__main__":
  app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e6a4-35-221-218-113.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Dec/2021 19:07:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2021 19:08:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2021 19:09:30] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2021 19:12:49] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2021 19:13:20] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Dec/2021 19:19:11] "POST / HTTP/1.1" 200 -


In [ ]:
train_list = []
for filename in os.listdir("drive/MyDrive/Colab Notebooks/Spectrograms/train_spectr")[:187]:
  train_list.append([filename.split(".")[0] + ".png", "amerob"])
file1 = open('train_data.csv', 'w+', newline = '')    
with file1:
    write = csv.writer(file1)
    write.writerows(train_list)


In [ ]:
train_list = []
for filename in os.listdir("drive/MyDrive/Colab Notebooks/Spectrograms/train_spectr")[187:]:
  train_list.append([filename.split(".")[0] + ".png", "blujay"])
with open("train_data.csv", "a") as file1:
    write = csv.writer(file1)
    write.writerows(train_list)

In [ ]:
train_list = []
for filename in os.listdir("drive/MyDrive/Colab Notebooks/Spectrograms/dump train"):
  train_list.append([filename.split(".")[0] + ".png", "norcar"])
with open("train_data.csv", "a") as file1:
    write = csv.writer(file1)
    write.writerows(train_list)

In [7]:
train_list = []
for filename in os.listdir("drive/MyDrive/Colab Notebooks/Spectrograms/dump train"):
  train_list.append([filename.split(".")[0] + ".png", "houspa"])
with open("drive/MyDrive/Colab Notebooks/train_data.csv", "a") as file1:
    write = csv.writer(file1)
    write.writerows(train_list)

In [32]:
test_list = []
for filename in os.listdir("drive/MyDrive/Colab Notebooks/Spectrograms/test_spectr")[:20]:
  test_list.append([filename.split(".")[0] + ".png", "amerob"])
file1 = open('drive/MyDrive/Colab Notebooks/test_data1.csv', 'w+', newline = '')    
with file1:
    write = csv.writer(file1)
    write.writerows(test_list)

In [33]:
test_list = []
for filename in os.listdir("drive/MyDrive/Colab Notebooks/Spectrograms/test_spectr")[20:40]:
  test_list.append([filename.split(".")[0] + ".png", "blujay"])
with open("drive/MyDrive/Colab Notebooks/test_data1.csv", "a") as file1:
    write = csv.writer(file1)
    write.writerows(test_list)

In [34]:
test_list = []
for filename in os.listdir("drive/MyDrive/Colab Notebooks/Spectrograms/test_spectr")[40:92]:
  test_list.append([filename.split(".")[0] + ".png", "norcar"])
with open("drive/MyDrive/Colab Notebooks/test_data1.csv", "a") as file1:
    write = csv.writer(file1)
    write.writerows(test_list)

In [35]:
test_list = []
for filename in os.listdir("drive/MyDrive/Colab Notebooks/Spectrograms/test_spectr")[92:]:
  test_list.append([filename.split(".")[0] + ".png", "houspa"])
with open("drive/MyDrive/Colab Notebooks/test_data1.csv", "a") as file1:
    write = csv.writer(file1)
    write.writerows(test_list)